# Quantum Buffon's Needle Problem Demonstration

## Abstract

Quantum computes promise numerous advantages, including quantum parallelism, quantum simulation, and quantum cryptograhpy.
Since the theory of local hidden variables has been disproven, it is established that quantum systems are capable of generating truly random outputs.
By preparing qubits in superposition states, one can obtain uniformly distributed random outputs.
This notebook presents a demonstration of quantum randomness through Buffon's needle problem, a probabilistic mehtod for estimating the value of $\pi$.
In this problem, needles of fixed length are randomly dropped onto a plane with evenly spaced parallel lines, and the ratio between the number of intersections and the total number of throws converges to $\pi$.
The random positions and orientations of the needles are generated using a quantum computer, effectively simulating the experiment.
The purpose of this work is not to accelerate the computation of $\pi$, but rather to illustrate the quality of randomness obtainable from quantum processes and to highlight their potential in probabilistic simulations.
This demonstration connects a mathematical conclusion to modern quantum technology, offering both pedagogical value and an example of the interplay between probability and quantum computation.

## Introduction

### Buffon's Needle Problem

We shall begin by introducing the context behind Buffon's needle problem.
Consider a plane marked with evenly spaced horizontal parallel lines, separated by distance $D$.
A set of needles, each of length fixed $l < D$, is randomly dropped onto the plane.
Some needles will intersect (or cross) one of the lines, while others will land entirely in the blank spaces between the lines.
If we let $N_{\text{cross}}$ denote the number of needles that intersect a line and $N_{\text{total}}$ the total number of needles thrown, then the ratio $\frac{N_{\text{cross}}}{N_{\text{total}}}$ converges, as the number of throws increases, to the true probability $\mathbb{P}_{\text{cross}}$ of a needle crossing a line.
Consequently, the probability can be approximated by:
$$\mathbb{P}_{\text{cross}} \approx \frac{N_{\text{cross}}}{N_{\text{total}}} \text{.}$$

Let us simplify the situation.
We can remove all the lines on the plane except for two consecutive ones separated by distance $D$.
Now place a needle between these lines with a random orientation $\theta$.

We can look at this problem from a different perspective.
Instead of focusing on the two lines, imagine throwing a line segment of length $D$ and checking whether it lands on top of the needle.
In this setting, the position (both horizontal and vertical) position of the needle does not matter; only the vertical projection of the needle, denoted $y$, is relevant.
By symmetry, it suffices to restrict the orientation to the range $0 \leq \theta \leq \frac{\pi}{2}$.

Once a line is thrown into the strip, the probability that it lands on the needle is $\frac{y}{D}$.
Since the vertical projection can be expressed in terms of the orientation, $y = l\sin(\theta)$, we have:
$$P(\theta) = \frac{y}{D} = \frac{l\sin(\theta)}{D} \text{.}$$

Here, $P(\theta)$ represents the conditional probability of a crossing given that the needle has orientation $\theta$.
However, in the actual experiment, the orientation $\theta$ itself is random and uniformly distributed in $\left[0, \frac{\pi}{2}\right]$.
To obtain the overall crossing probability, we therefore compute the expected value of $P(\theta)$ over all possible orientations in $\left[0, \frac{\pi}{2}\right]$.
$$
\begin{align*}
    \mathbb{P}_{\text{cross}} &= \mathbb{E}[P(\theta)] \\
    &= \frac{1}{\frac{\pi}{2}-0} \int_{0}^{\frac{\pi}{2}} P(\theta) \,\text{d}\theta \\
    &= \frac{2}{\pi} \int_{0}^{\frac{\pi}{2}} \frac{l\sin(\theta)}{D} \,\text{d}\theta \\
    &= \frac{2l}{\pi{D}} \left[-\cos(\theta)\right]_{\theta=0}^{\theta=\frac{\pi}{2}} \\
    &= \frac{2l}{\pi{D}}
\end{align*}
$$

From the theoratical probability we just derived, we can now construct a formula to approximate the value of $\pi$.
$$
\begin{align*}
    \mathbb{P}_{\text{cross}} \approx \frac{N_{\text{cross}}}{N_{\text{total}}} &\implies \frac{2l}{\pi{D}} \approx \frac{N_{\text{cross}}}{N_{\text{total}}} \\
    &\implies \frac{1}{\pi} \approx \frac{DN_{\text{cross}}}{2lN_{\text{total}}} \\
    &\implies \pi \approx \frac{2lN_{\text{all}}}{DN_{\text{cross}}}
\end{align*}
$$

To simulate this problem, we need to randomly generate two numbers: the vertical position of the thrown line, $y' \in [0, D]$, and the orientation of the needle, $\theta \in \left[0, \frac{\pi}{2}\right]$.
A crossing is detected by cehcking whether $y' \leq y = l\sin(\theta)$.

Next, we shall describe how to generate uniformly random bits using a quantum computer and how to convert the retrieved bit strings into uniformly distributed floating-point numbers within a given range.

### Random Number Generation

## Demonstration

### Installation

Assuming that Python is installed on the hosted device, we now install the essential packages for this demonstration: Qiskit and Qiskit Aer.

In [ ]:
%pip install qiskit qiskit-aer qiskit-ibm-runtime matplotlib tqdm -U

In [ ]:
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_aer.primitives import SamplerV2 as Simulator
import math
import matplotlib.pyplot as plt
from tqdm import tqdm

### Classical Simulation

It is good idea to simulate first and then run the algorithm in a real quantum computer.

In [ ]:
def get_qc(num_qubits: int) -> QuantumCircuit:
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        qc.h(i)
    qc.measure_all()
    
    return qc

In [ ]:
def get_counts_simulation(qc: QuantumCircuit, throws: int) -> dict:
    simulator = Simulator()
    job = simulator.run([qc], shots=2*throws)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    return counts

In [ ]:
def get_binary_string(counts: dict) -> str:
    picked = list(counts.keys())[0]

    if counts[picked] == 1:
        del counts[picked]
    else:
        counts[picked] -= 1

    return picked

In [ ]:
def get_float(string: str, start: float, end: float) -> float:
    integer = int(string, 2)
    normalized = integer / (2 ** len(string) - 1)
    scale = end - start

    return start + scale * normalized

In [ ]:
def throw_needle(needle_length: float, position: float, orientation: float) -> bool:
    return position <= needle_length * math.sin(orientation)

In [ ]:
strip_length = 5.0 # Your strip length here.
needle_length = 3.0 # Your needle length here.
num_qubits = 52 # Number of qubits to use.
throws = 50_000 # Number of needle throws.

if strip_length <= needle_length:
    raise ValueError("Needle length must be less than strip length.")

In [ ]:
qc = get_qc(num_qubits)
counts = get_counts_simulation(qc, throws)

cross, total = 0, 0
throw_list_simulation, error_list_simulation = [], []
report_string = ""
width = len(str(throws))

for i in tqdm(range(1, throws+1)):
    position = get_float(get_binary_string(counts), 0.0, strip_length)
    orientation = get_float(get_binary_string(counts), 0.0, math.pi/2.0)

    cross += 1 if throw_needle(needle_length, position, orientation) else 0
    total += 1

    if cross != 0:
        approx_pi = (2.0 * needle_length * total) / (strip_length * cross)
        error = abs(math.pi - approx_pi) / math.pi
        error_rate = error * 100.0

        throw_list_simulation.append(i)
        error_list_simulation.append(error)

    if i % (throws // 20) == 0:
        report_string += f"Throw #{i:{width}}: π ≈ {approx_pi:.10f} (error rate: {error_rate:.10f}%)\n"

print(report_string)

plt.figure()
plt.loglog(throw_list_simulation, error_list_simulation, linestyle='-', color='blue')
plt.xlabel("Number of Throws")
plt.ylabel("Error Rate (%)")
plt.title("Convergence of Buffon's Needle Simulation (Simulation)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

### Quantum Implementation

In [ ]:
def get_counts_quantum(qc: QuantumCircuit, throws: int, token: str, crn: str) -> dict:
    service = QiskitRuntimeService(channel='ibm_quantum_platform', token=token, instance=crn)
    backend = service.backends(simulator=False, operational=True)[0]

    pm = generate_preset_pass_manager(backend=backend, optimization_level=0)
    trans_qc = pm.run(qc)
    
    sampler = Sampler(mode=backend)
    job = sampler.run([trans_qc], shots=2*throws)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    return counts

In [ ]:
token = "" # Your IBM Quantum token here.
crn = "" # Your IBM Quantum cloud resource name here.

In [ ]:
qc = get_qc(num_qubits)
counts = get_counts_quantum(qc, throws, token, crn)

cross, total = 0, 0
throw_list_quantum, error_list_quantum = [], []
report_string = ""
width = len(str(throws))

for i in tqdm(range(1, throws+1)):
    position = get_float(get_binary_string(counts), 0.0, strip_length)
    orientation = get_float(get_binary_string(counts), 0.0, math.pi/2.0)

    cross += 1 if throw_needle(needle_length, position, orientation) else 0
    total += 1

    if cross != 0:
        approx_pi = (2.0 * needle_length * total) / (strip_length * cross)
        error = abs(math.pi - approx_pi) / math.pi
        error_rate = error * 100.0

        throw_list_quantum.append(i)
        error_list_quantum.append(error)

    if i % (throws // 20) == 0:
        report_string += f"Throw #{i:{width}}: π ≈ {approx_pi:.10f} (error rate: {error_rate:.10f}%)\n"

print(report_string)

plt.figure()
plt.loglog(throw_list_quantum, error_list_quantum, linestyle='-', color='red')
plt.xlabel("Number of Throws")
plt.ylabel("Error Rate (%)")
plt.title("Convergence of Buffon's Needle Simulation (Quantum)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

## Discussion

In fact this is not a good demonstration for the efficiency of using quantum computers.

## Conclusion


## References

* https://mathworld.wolfram.com/BuffonsNeedleProblem.html